<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Разбиение-данных" data-toc-modified-id="Разбиение-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбиение данных</a></span></li><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Решающее дерево</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Лес" data-toc-modified-id="Лес-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Лес</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Логистическая регрессия</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# ML-модель для выбора нового тарифа

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются
архивными тарифами.

Нужно построить модель для задачи классификации, которая выберет подходящий
тариф. Цель: добавиться accuracy >= 0.75

## Подготовка данных

In [65]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from tqdm.contrib.itertools import product
from tqdm.notebook import tqdm

In [66]:
def read_file(name: str):
    # try open local file first then — if failed — open on server
    try:
        return pd.read_csv(f'./datasets/{name}.csv', sep=',')
    except FileNotFoundError:
        try:
            return pd.read_csv(f'/datasets/{name}.csv', sep=',')
        except Exception as e:
            print('Не удалось считать файл с данными на сервере', e, sep='\n')
            return None
        
    except Exception as e:
        print('Не удалось считать файл с данными на диске', e, sep='\n')
    return None

In [67]:
df = read_file('users_behavior')

Посмотрим на случайные 7 записей в таблице

In [68]:
df.head(7)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1


Посмотрим на типы данных и количество значений

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропущенных нет, проставим верные типы

In [70]:
df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')

## Разбиение данных

Поделим имеющиеся данные на тестовую, валидационную и обучающие выборки
по правилу 60—20—20:

In [71]:
random_state = 12345

In [72]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=random_state)

features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=random_state)

## Решающее дерево

Найдем лучшую модель со следующими гиперпараметрами:
- глубина 1…20;
- критерий gini или entropy;
- минимальное количество объектов в узле 2…100 с шагом в 10

In [73]:
best_accuracy= 0
best_model = None

for params in product(
        ['gini', 'entropy'],
        range(1, 21),
        range(2, 101, 1)):
    criterion, max_depth, min_samples_split = params
    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=random_state)
    
    model.fit(features_train, target_train)
    
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

  0%|          | 0/3960 [00:00<?, ?it/s]

In [74]:
print('Лучшая глубина:', best_model.max_depth)
print('Лучший критерий:', best_model.criterion)
print('Минимальный размер узла:', best_model.min_samples_split)
print(f'Точность: {best_accuracy:.3f}')

Лучшая глубина: 14
Лучший критерий: entropy
Минимальный размер узла: 70
Точность: 0.795


Обучим лучшую модель на объединенной выборке

In [75]:
best_model.fit(
    pd.concat([features_train, features_valid], ignore_index=True),
    pd.concat([target_train, target_valid], ignore_index=True));

Посмотрим, на качество лучшей модели на тестовой выборке:

In [76]:
predictions = best_model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)

In [77]:
print(f'Качество дерева на тестовой выборке: {accuracy: .3f}')

Качество дерева на тестовой выборке:  0.776


### Вывод

Видно, что качество модели растет с увеличенным до 70 по сравнению со стандартным
2 минимальным размером узла.

Лучшее качество показал критерий entropy и глубина в 14 (при дальнейшем
увеличении глубины, качество не улучшалось).


## Лес

Найдем лучшую модель со следующими гиперпараметрами:
- количество деревьев 100…200 с шагом 2;
- критерий gini или entropy.

In [78]:
best_accuracy= 0
best_model = None

for params in product(
        ['gini', 'entropy'],
        range(100, 201, 2)):
    criterion, n_estimators = params
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        n_jobs=8,
        random_state=random_state)
    model.fit(features_train, target_train)
    
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

  0%|          | 0/102 [00:00<?, ?it/s]

In [79]:
print('Лучшая глубина:', best_model.n_estimators)
print('Лучший критерий:', best_model.criterion)
print(f'Точность: {best_accuracy:.3f}')

Лучшая глубина: 120
Лучший критерий: entropy
Точность: 0.804


Обучим лучшую модель на объединенной выборке

In [80]:
best_model.fit(
    pd.concat([features_train, features_valid], ignore_index=True),
    pd.concat([target_train, target_valid], ignore_index=True));

Посмотрим, на качество лучшей модели на тестовой выборке:

In [81]:
predictions = best_model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)

In [82]:
print(f'Качество дерева на тестовой выборке: {accuracy: .3f}')

Качество дерева на тестовой выборке:  0.781


### Вывод

Лучшее качество показал критерий entropy.

При глубине свыше 150 качество не растет.

Качество на валидационной и тестовой выборке немного лучше простого дерева.

## Логистическая регрессия

Найдем лучшую модель

In [83]:
best_accuracy= 0
best_model = None

for max_iter in tqdm(range(100, 1000, 100)):
    model = LogisticRegression(
        max_iter=max_iter,
        multi_class='ovr',
        n_jobs=8,
        random_state=random_state)
    model.fit(features_train, target_train)
    
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

  0%|          | 0/9 [00:00<?, ?it/s]

In [84]:
print('Лучшее количество итераций:', best_model.max_iter)
print(f'Точность: {best_accuracy:.3f}')

Лучшее количество итераций: 100
Точность: 0.726


Обучим лучшую модель на объединенной выборке

In [85]:
best_model.fit(
    pd.concat([features_train, features_valid], ignore_index=True),
    pd.concat([target_train, target_valid], ignore_index=True));

Посмотрим, на качество лучшей модели на тестовой выборке:

In [86]:
predictions = best_model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)

In [87]:
print(f'Качество регресии на тестовой выборке: {accuracy: .3f}')

Качество регресии на тестовой выборке:  0.757


### Вывод

Рост количества итераций не играл роли. Качество получилось
самым плохим по сравнению с двумя другими моделями: примерно на
5% ниже.

## Общий вывод

На валидационной выборке получили следующий топ:
- лес 0,804
- дерево 0,795
- регрессия 0,726

На тестовой выборке:
- лес 0,781
- дерево 0,776
- регрессия 0,757

В теории качество должно быть в порядке лес → регрессия → дерево,
но у нас получилось поднять дерево выше регрессии и практически на уровень
леса за счет подбора трех гиперпараметров.